In [ ]:
# 必要なライブラリをインポート
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import logging
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sqlalchemy import create_engine, text
import torch.nn.functional as F
import japanize_matplotlib

# 自作モジュールをインポート
from _dataset import HorguesDataset
from _models import HorguesModel
from horgues3.losses import HorguesLoss
from horgues3.odds import get_odds_dataframes
from horgues3.haraimodoshi import get_haraimodoshi_dataframes
from horgues3.database import create_database_engine

In [ ]:
# ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# デバイス設定
device = torch.device('cuda:1')
logger.info(f"使用デバイス: {device}")

In [ ]:
# 学習パラメータ設定
BATCH_SIZE = 128

In [ ]:
checkpoint = torch.load('outputs/best_model.pth', map_location='cpu')

In [ ]:
# 学習期間・検証期間設定
START_DATE = '20230101'
END_DATE = '20231231'

logger.info(f"期間: {START_DATE} - {END_DATE}")

In [ ]:
# データセットの作成
logger.info("データセットを作成中...")
dataset = HorguesDataset(
    start_date=START_DATE,
    end_date=END_DATE,
    preprocessing_params=checkpoint['preprocessing_params'],
    cache_dir='cache/test',
    use_cache=True
)

logger.info(f"データサイズ: {len(dataset)}")

In [ ]:
loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [ ]:
# モデル設定の取得
model_config = checkpoint['model_config']
logger.info(f"数値特徴量数: {len(model_config['numerical_features'])}")
logger.info(f"カテゴリ特徴量数: {len(model_config['categorical_features'])}")
logger.info(f"時系列データ: {model_config['sequence_names']}")

In [ ]:
# モデルの作成
model = HorguesModel(
    sequence_names=model_config['sequence_names'],
    feature_aliases=model_config['feature_aliases'],
    numerical_features=model_config['numerical_features'],
    categorical_features=model_config['categorical_features'],
).to(device)

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
race_ids = []
masks = []
targets = []
scores = []

model.eval()
with torch.no_grad():
    pbar = tqdm(loader, desc='Predict')
    for batch in pbar:
        race_ids.extend(batch['race_id'])
        masks.append(batch['mask'])
        targets.append(batch['target'])

        x_num = {k: v.to(device) for k, v in batch['x_num'].items()}
        x_cat = {k: v.to(device) for k, v in batch['x_cat'].items()}
        sequence_data = {}
        for seq_name, seq_data in batch['sequence_data'].items():
            sequence_data[seq_name] = {
                'x_num': {k: v.to(device) for k, v in seq_data['x_num'].items()},
                'x_cat': {k: v.to(device) for k, v in seq_data['x_cat'].items()},
                'mask': seq_data['mask'].to(device)
            }
        mask = batch['mask'].to(device)
        target = batch['target'].to(device)

        scores.append(model(x_num, x_cat, sequence_data, mask).cpu())

    masks = torch.cat(masks, dim=0)
    targets = torch.cat(targets, dim=0)
    scores = torch.cat(scores, dim=0)

In [ ]:
odds = get_odds_dataframes(START_DATE, END_DATE)
haraimodoshi = get_haraimodoshi_dataframes(START_DATE, END_DATE)

In [ ]:
def convert_to_series(tensor, name):
    tmp = pd.DataFrame(tensor, index=race_ids, columns=np.arange(1, 19))
    tmp = tmp.reset_index(names='race_id').melt(id_vars='race_id', var_name='uma_number')
    tmp = tmp.set_index(['race_id', 'uma_number']).value.rename(name)
    return tmp

In [ ]:
df = pd.DataFrame(
    {
        'score_0': convert_to_series(F.sigmoid(scores[:, :, 0]), 'score_0'),
        'score_1': convert_to_series(F.sigmoid(scores[:, :, 1]), 'score_1'),
        'score_2': convert_to_series(F.sigmoid(scores[:, :, 2]), 'score_2'),
        'target_0': convert_to_series(targets[:, :, 0], 'target_0'),
        'target_1': convert_to_series(targets[:, :, 1], 'target_1'),
        'target_2': convert_to_series(targets[:, :, 2], 'target_2'),
        'mask': convert_to_series(masks, 'mask'),
    }
)
df = df[df['mask']].sort_index()

In [ ]:
df['score_ranking_0'] = df.groupby('race_id')['score_0'].rank(method='first', ascending=False)
df['score_ranking_1'] = df.groupby('race_id')['score_1'].rank(method='first', ascending=False)
df['score_ranking_2'] = df.groupby('race_id')['score_2'].rank(method='first', ascending=False)

In [ ]:
# 全馬券種購入シミュレーション
from itertools import product, combinations, permutations
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
import multiprocessing as mp
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

def simulate_tansho_betting(df, odds, haraimodoshi, thresholds):
    """
    単勝購入シミュレーション
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 1着候補の選定
        candidates = race_df[
            (race_df['score_0'] >= thresholds['score_0']) & 
            (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])
        ].index.values
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        for horse in candidates:
            horse_str = f"{horse:02d}"
            
            # オッズをチェック
            if race_id in odds['tansho'].index and horse_str in odds['tansho'].columns:
                odds_value = odds['tansho'].loc[race_id, horse_str]
                if odds_value >= thresholds['min_odds']:
                    total_bet += 100
                    
                    # 的中チェック
                    if race_id in haraimodoshi['tansho'].index:
                        payout_value = haraimodoshi['tansho'].loc[race_id, horse_str]
                        if payout_value > 0:
                            payout += payout_value
                            hit_count += 1
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(candidates)
            })
    
    return pd.DataFrame(results)

def simulate_fukusho_betting(df, odds, haraimodoshi, thresholds):
    """
    複勝購入シミュレーション
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 3着以内候補の選定（複勝は1-3着全ての予測を考慮）
        candidates = race_df[
            ((race_df['score_0'] >= thresholds['score_0']) & (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])) |
            ((race_df['score_1'] >= thresholds['score_1']) & (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])) |
            ((race_df['score_2'] >= thresholds['score_2']) & (race_df['score_ranking_2'] <= thresholds['score_ranking_2']))
        ].index.values
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        for horse in candidates:
            horse_str = f"{horse:02d}"
            
            # オッズをチェック
            if race_id in odds['fukusho'].index and horse_str in odds['fukusho'].columns:
                odds_value = odds['fukusho'].loc[race_id, horse_str]
                if odds_value >= thresholds['min_odds']:
                    total_bet += 100
                    
                    # 的中チェック
                    if race_id in haraimodoshi['fukusho'].index:
                        payout_value = haraimodoshi['fukusho'].loc[race_id, horse_str]
                        if payout_value > 0:
                            payout += payout_value
                            hit_count += 1
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(candidates)
            })
    
    return pd.DataFrame(results)

def simulate_umaren_betting(df, odds, haraimodoshi, thresholds):
    """
    馬連購入シミュレーション（BOX買い）
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 1着候補と2着候補の選定
        first_candidates = race_df[
            (race_df['score_0'] >= thresholds['score_0']) & 
            (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])
        ].index.values
        
        second_candidates = race_df[
            (race_df['score_1'] >= thresholds['score_1']) & 
            (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])
        ].index.values
        
        # BOX買いの候補（1着または2着候補）
        all_candidates = list(set(list(first_candidates) + list(second_candidates)))
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        # 2頭BOXの組み合わせ
        for horse1, horse2 in combinations(all_candidates, 2):
            combination = f"{min(horse1, horse2):02d}-{max(horse1, horse2):02d}"
            
            # オッズをチェック
            if race_id in odds['umaren'].index and combination in odds['umaren'].columns:
                odds_value = odds['umaren'].loc[race_id, combination]
                if odds_value >= thresholds['min_odds']:
                    total_bet += 100
                    
                    # 的中チェック
                    if race_id in haraimodoshi['umaren'].index:
                        payout_value = haraimodoshi['umaren'].loc[race_id, combination]
                        if payout_value > 0:
                            payout = payout_value
                            hit_count = 1
                            break
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(list(combinations(all_candidates, 2)))
            })
    
    return pd.DataFrame(results)

def simulate_wide_betting(df, odds, haraimodoshi, thresholds):
    """
    ワイド購入シミュレーション（BOX買い）
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 3着以内候補の選定
        candidates = race_df[
            ((race_df['score_0'] >= thresholds['score_0']) & (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])) |
            ((race_df['score_1'] >= thresholds['score_1']) & (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])) |
            ((race_df['score_2'] >= thresholds['score_2']) & (race_df['score_ranking_2'] <= thresholds['score_ranking_2']))
        ].index.values
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        # 2頭BOXの組み合わせ
        for horse1, horse2 in combinations(candidates, 2):
            combination = f"{min(horse1, horse2):02d}-{max(horse1, horse2):02d}"
            
            # オッズをチェック
            if race_id in odds['wide'].index and combination in odds['wide'].columns:
                odds_value = odds['wide'].loc[race_id, combination]
                if odds_value >= thresholds['min_odds']:
                    total_bet += 100
                    
                    # 的中チェック
                    if race_id in haraimodoshi['wide'].index:
                        payout_value = haraimodoshi['wide'].loc[race_id, combination]
                        if payout_value > 0:
                            payout += payout_value  # ワイドは複数的中の可能性があるので加算
                            hit_count += 1
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(list(combinations(candidates, 2)))
            })
    
    return pd.DataFrame(results)

def simulate_umatan_betting(df, odds, haraimodoshi, thresholds):
    """
    馬単購入シミュレーション（フォーメーション）
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 1着候補と2着候補の選定
        first_candidates = race_df[
            (race_df['score_0'] >= thresholds['score_0']) & 
            (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])
        ].index.values
        
        second_candidates = race_df[
            (race_df['score_1'] >= thresholds['score_1']) & 
            (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])
        ].index.values
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        # フォーメーション購入
        for first, second in product(first_candidates, second_candidates):
            if first != second:
                combination = f"{first:02d}-{second:02d}"
                
                # オッズをチェック
                if race_id in odds['umatan'].index and combination in odds['umatan'].columns:
                    odds_value = odds['umatan'].loc[race_id, combination]
                    if odds_value >= thresholds['min_odds']:
                        total_bet += 100
                        
                        # 的中チェック
                        if race_id in haraimodoshi['umatan'].index:
                            payout_value = haraimodoshi['umatan'].loc[race_id, combination]
                            if payout_value > 0:
                                payout = payout_value
                                hit_count = 1
                                break
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len([1 for first, second in product(first_candidates, second_candidates) if first != second])
            })
    
    return pd.DataFrame(results)

def simulate_sanrenpuku_betting(df, odds, haraimodoshi, thresholds):
    """
    3連複購入シミュレーション（BOX買い）
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 3着以内候補の選定
        candidates = race_df[
            ((race_df['score_0'] >= thresholds['score_0']) & (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])) |
            ((race_df['score_1'] >= thresholds['score_1']) & (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])) |
            ((race_df['score_2'] >= thresholds['score_2']) & (race_df['score_ranking_2'] <= thresholds['score_ranking_2']))
        ].index.values
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        # 3頭BOXの組み合わせ
        for horse1, horse2, horse3 in combinations(candidates, 3):
            combination = f"{horse1:02d}-{horse2:02d}-{horse3:02d}"
            
            # オッズをチェック
            if race_id in odds['sanrenpuku'].index and combination in odds['sanrenpuku'].columns:
                odds_value = odds['sanrenpuku'].loc[race_id, combination]
                if odds_value >= thresholds['min_odds']:
                    total_bet += 100
                    
                    # 的中チェック
                    if race_id in haraimodoshi['sanrenpuku'].index:
                        payout_value = haraimodoshi['sanrenpuku'].loc[race_id, combination]
                        if payout_value > 0:
                            payout = payout_value
                            hit_count = 1
                            break
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(list(combinations(candidates, 3)))
            })
    
    return pd.DataFrame(results)
    
# 元の三連単シミュレーション関数をここに再定義（インポートできない場合）
def simulate_sanrentan_formation(df, odds, haraimodoshi, thresholds):
    """
    三連単フォーメーション購入シミュレーション（再定義版）
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 1着候補の選定
        first_candidates = race_df[
            (race_df['score_0'] >= thresholds['score_0']) & 
            (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])
        ].index.values
        
        # 2着候補の選定
        second_candidates = race_df[
            (race_df['score_1'] >= thresholds['score_1']) & 
            (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])
        ].index.values
        
        # 3着候補の選定
        third_candidates = race_df[
            (race_df['score_2'] >= thresholds['score_2']) & 
            (race_df['score_ranking_2'] <= thresholds['score_ranking_2'])
        ].index.values
        
        # フォーメーション購入対象の組み合わせ生成
        purchases = []
        total_bet = 0
        
        for first, second, third in product(first_candidates, second_candidates, third_candidates):
            if first != second and second != third and first != third:
                # 組み合わせ文字列を作成
                combination = f"{first:02d}-{second:02d}-{third:02d}"
                
                # オッズをチェック
                if race_id in odds['sanrentan'].index:
                    if combination in odds['sanrentan'].columns:
                        odds_value = odds['sanrentan'].loc[race_id, combination]
                        if odds_value >= thresholds['min_odds']:
                            purchases.append(combination)
                            total_bet += 100  # 100円購入と仮定
        
        # 的中チェック
        payout = 0
        hit_count = 0
        if race_id in haraimodoshi['sanrentan'].index and purchases:
            for combination in purchases:
                if combination in haraimodoshi['sanrentan'].columns:
                    payout_value = haraimodoshi['sanrentan'].loc[race_id, combination]
                    if payout_value > 0:
                        payout = payout_value
                        hit_count = 1
                        break
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(purchases)
            })
    
    return pd.DataFrame(results)


def simulate_all_betting_types(df, odds, haraimodoshi, thresholds):
    """
    全馬券種のシミュレーションを実行
    """
    simulators = {
        'tansho': simulate_tansho_betting,
        'fukusho': simulate_fukusho_betting,
        'umaren': simulate_umaren_betting,
        'wide': simulate_wide_betting,
        'umatan': simulate_umatan_betting,
        'sanrenpuku': simulate_sanrenpuku_betting,
        'sanrentan': simulate_sanrentan_formation,  # 既存の関数を使用
    }
    
    results = {}
    
    for bet_type, simulator in simulators.items():
        try:
            sim_results = simulator(df, odds, haraimodoshi, thresholds)
            
            if len(sim_results) == 0:
                result = {
                    'total_bet': 0,
                    'total_payout': 0,
                    'hit_count': 0,
                    'recovery_rate': 0,
                    'max_payout': 0,
                    'evaluation_score': 0,
                    'race_count': 0
                }
            else:
                total_bet = sim_results['total_bet'].sum()
                total_payout = sim_results['payout'].sum()
                hit_count = sim_results['hit_count'].sum()
                max_payout = sim_results['payout'].max()
                
                # 回収率計算
                recovery_rate = total_payout / total_bet if total_bet > 0 else 0
                
                # 評価値計算
                if hit_count > 0 and total_bet > 0:
                    adjusted_recovery_rate = (total_payout - max_payout) / total_bet
                    evaluation_score = adjusted_recovery_rate ** hit_count
                else:
                    evaluation_score = 0
                
                result = {
                    'total_bet': total_bet,
                    'total_payout': total_payout,
                    'hit_count': hit_count,
                    'recovery_rate': recovery_rate,
                    'max_payout': max_payout,
                    'evaluation_score': evaluation_score,
                    'race_count': len(sim_results)
                }
            
            results[bet_type] = result
            
        except Exception as e:
            print(f"Error in {bet_type} simulation: {e}")
            results[bet_type] = {
                'total_bet': 0,
                'total_payout': 0,
                'hit_count': 0,
                'recovery_rate': 0,
                'max_payout': 0,
                'evaluation_score': 0,
                'race_count': 0
            }
    
    return results

# 効率化された馬券種別閾値組み合わせ生成関数
def create_optimized_threshold_combinations_by_betting_type():
    """
    馬券種ごに必要な閾値のみを生成して効率化
    """
    score_candidates = [0.1, 0.15, 0.2, 0.25, 0.3]
    ranking_candidates = [1, 18]
    min_odds_candidates = [1.5, 2, 3, 4, 5.0, 7.5, 10.0, 15.0, 20.0, 30.0, 40, 50]
    
    betting_type_configs = {
        'tansho': {
            'score_vars': ['score_0'],  # 1着のみ
            'ranking_vars': ['score_ranking_0']
        },
        'fukusho': {
            'score_vars': ['score_2'],  # 3着以内（最低スコア）
            'ranking_vars': ['score_ranking_2']
        },
        'umaren': {
            'score_vars': ['score_2'],  # 2着以内（最低スコア）
            'ranking_vars': ['score_ranking_2']
        },
        'wide': {
            'score_vars': ['score_2'],  # 3着以内（最低スコア）
            'ranking_vars': ['score_ranking_2']
        },
        'umatan': {
            'score_vars': ['score_0', 'score_1'],  # 1着、2着
            'ranking_vars': ['score_ranking_0', 'score_ranking_1']
        },
        'sanrenpuku': {
            'score_vars': ['score_2'],  # 3着以内（最低スコア）
            'ranking_vars': ['score_ranking_2']
        },
        'sanrentan': {
            'score_vars': ['score_0', 'score_1', 'score_2'],  # 1着、2着、3着
            'ranking_vars': ['score_ranking_0', 'score_ranking_1', 'score_ranking_2']
        }
    }
    
    all_combinations = {}
    
    for bet_type, config in betting_type_configs.items():
        combinations = []
        
        # 必要なスコア変数の組み合わせを生成
        score_vars = config['score_vars']
        ranking_vars = config['ranking_vars']
        
        if len(score_vars) == 1:
            # 単一スコア（単勝、複勝、馬連、ワイド、3連複）
            for score in score_candidates:
                for ranking in ranking_candidates:
                    for min_odds in min_odds_candidates:
                        thresholds = {
                            'min_odds': min_odds
                        }
                        # 全てのスコア変数を設定（使用しないものはダミー値）
                        thresholds['score_0'] = score if 'score_0' in score_vars else 0.1
                        thresholds['score_1'] = score if 'score_1' in score_vars else 0.1
                        thresholds['score_2'] = score if 'score_2' in score_vars else 0.1
                        thresholds['score_ranking_0'] = ranking if 'score_ranking_0' in ranking_vars else 18
                        thresholds['score_ranking_1'] = ranking if 'score_ranking_1' in ranking_vars else 18
                        thresholds['score_ranking_2'] = ranking if 'score_ranking_2' in ranking_vars else 18
                        combinations.append(thresholds)
                        
        elif len(score_vars) == 2:
            # 2つのスコア（馬単）
            for score_0 in score_candidates:
                for score_1 in score_candidates:
                    for ranking_0 in ranking_candidates:
                        for ranking_1 in ranking_candidates:
                            for min_odds in min_odds_candidates:
                                thresholds = {
                                    'score_0': score_0,
                                    'score_1': score_1,
                                    'score_2': 0.1,  # ダミー値
                                    'score_ranking_0': ranking_0,
                                    'score_ranking_1': ranking_1,
                                    'score_ranking_2': 18,  # ダミー値
                                    'min_odds': min_odds
                                }
                                combinations.append(thresholds)
                                
        else:
            # 3つのスコア（3連単）
            for score_0 in score_candidates:
                for score_1 in score_candidates:
                    for score_2 in score_candidates:
                        for ranking_0 in ranking_candidates:
                            for ranking_1 in ranking_candidates:
                                for ranking_2 in ranking_candidates:
                                    for min_odds in min_odds_candidates:
                                        thresholds = {
                                            'score_0': score_0,
                                            'score_1': score_1,
                                            'score_2': score_2,
                                            'score_ranking_0': ranking_0,
                                            'score_ranking_1': ranking_1,
                                            'score_ranking_2': ranking_2,
                                            'min_odds': min_odds
                                        }
                                        combinations.append(thresholds)
        
        all_combinations[bet_type] = combinations
        print(f"{bet_type}: {len(combinations):,} combinations")
    
    return all_combinations

def evaluate_optimized_betting_types_worker(args):
    """
    最適化された馬券種評価のワーカー関数
    """
    bet_type, thresholds, df, odds, haraimodoshi = args
    
    simulators = {
        'tansho': simulate_tansho_betting,
        'fukusho': simulate_fukusho_betting,
        'umaren': simulate_umaren_betting,
        'wide': simulate_wide_betting,
        'umatan': simulate_umatan_betting,
        'sanrenpuku': simulate_sanrenpuku_betting,
        'sanrentan': simulate_sanrentan_formation,
    }
    
    try:
        simulator = simulators[bet_type]
        sim_results = simulator(df, odds, haraimodoshi, thresholds)
        
        if len(sim_results) == 0:
            result = {
                'total_bet': 0,
                'total_payout': 0,
                'hit_count': 0,
                'recovery_rate': 0,
                'max_payout': 0,
                'evaluation_score': 0,
                'race_count': 0
            }
        else:
            total_bet = sim_results['total_bet'].sum()
            total_payout = sim_results['payout'].sum()
            hit_count = sim_results['hit_count'].sum()
            max_payout = sim_results['payout'].max()
            
            # 回収率計算
            recovery_rate = total_payout / total_bet if total_bet > 0 else 0
            
            # 評価値計算
            if hit_count > 0 and total_bet > 0:
                adjusted_recovery_rate = (total_payout - max_payout) / total_bet
                evaluation_score = adjusted_recovery_rate ** hit_count
            else:
                evaluation_score = 0
            
            result = {
                'total_bet': total_bet,
                'total_payout': total_payout,
                'hit_count': hit_count,
                'recovery_rate': recovery_rate,
                'max_payout': max_payout,
                'evaluation_score': evaluation_score,
                'race_count': len(sim_results)
            }
        
        # 閾値情報を結果に追加
        result.update(thresholds)
        return bet_type, result
        
    except Exception as e:
        print(f"Error in {bet_type} simulation: {e}")
        # エラーの場合はダミーの結果を返す
        empty_result = {
            'total_bet': 0,
            'total_payout': 0,
            'hit_count': 0,
            'recovery_rate': 0,
            'max_payout': 0,
            'evaluation_score': 0,
            'race_count': 0
        }
        empty_result.update(thresholds)
        return bet_type, empty_result

def grid_search_optimized_betting_types(df, odds, haraimodoshi, n_workers=None):
    """
    最適化された全馬券種の並列グリッドサーチ
    """
    if n_workers is None:
        n_workers = min(mp.cpu_count(), 8)
    
    print(f"使用プロセス数: {n_workers}")
    
    # 馬券種別の閾値組み合わせを生成
    all_combinations = create_optimized_threshold_combinations_by_betting_type()
    
    # 総組み合わせ数を計算
    total_combinations = sum(len(combinations) for combinations in all_combinations.values())
    print(f"総組み合わせ数: {total_combinations:,}")
    
    # 引数リストを作成
    args_list = []
    for bet_type, combinations in all_combinations.items():
        for thresholds in combinations:
            args_list.append((bet_type, thresholds, df, odds, haraimodoshi))
    
    # 各馬券種の最良結果を追跡
    bet_types = list(all_combinations.keys())
    best_scores = {bet_type: -1 for bet_type in bet_types}
    best_thresholds = {bet_type: None for bet_type in bet_types}
    best_results = {bet_type: None for bet_type in bet_types}
    all_results = {bet_type: [] for bet_type in bet_types}
    
    start_time = time.time()
    
    # 並列処理で実行
    with ProcessPoolExecutor(max_workers=n_workers) as executor:
        future_to_args = {executor.submit(evaluate_optimized_betting_types_worker, args): args for args in args_list}
        
        with tqdm(total=total_combinations, desc='Optimized Grid Search', 
                 bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]') as pbar:
            
            completed_count = 0
            for future in as_completed(future_to_args):
                try:
                    bet_type, result = future.result()
                    all_results[bet_type].append(result)
                    
                    # 最良の結果を更新
                    if result['evaluation_score'] > best_scores[bet_type]:
                        best_scores[bet_type] = result['evaluation_score']
                        best_thresholds[bet_type] = {k: result[k] for k in ['score_0', 'score_1', 'score_2', 
                                                                           'score_ranking_0', 'score_ranking_1', 
                                                                           'score_ranking_2', 'min_odds']}
                        best_results[bet_type] = result.copy()
                    
                    completed_count += 1
                    pbar.update(1)
                    
                    # 10%ごとに中間結果を表示
                    if completed_count % (total_combinations // 10) == 0:
                        elapsed_time = time.time() - start_time
                        progress_pct = (completed_count / total_combinations) * 100
                        pbar.set_postfix({
                            'Progress': f'{progress_pct:.1f}%',
                            'Elapsed': f'{elapsed_time:.1f}s'
                        })
                        
                except Exception as e:
                    print(f"タスクでエラーが発生: {e}")
                    pbar.update(1)
    
    elapsed_time = time.time() - start_time
    print(f"\n並列処理完了時間: {elapsed_time:.2f}秒")
    print(f"処理効率: {total_combinations / elapsed_time:.1f} combinations/sec")
    
    # DataFrameに変換
    all_results_df = {}
    for bet_type in bet_types:
        all_results_df[bet_type] = pd.DataFrame(all_results[bet_type])
    
    return best_thresholds, best_results, all_results_df

def plot_all_betting_types_results(best_results, all_results_df):
    """
    全馬券種の結果をプロット
    """
    bet_types = list(best_results.keys())
    
    # 1. 回収率比較
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('All Betting Types Analysis', fontsize=16)
    
    # 回収率
    ax1 = axes[0, 0]
    recovery_rates = [best_results[bt]['recovery_rate'] for bt in bet_types]
    bars1 = ax1.bar(bet_types, recovery_rates, alpha=0.7)
    ax1.set_ylabel('Recovery Rate')
    ax1.set_title('Best Recovery Rate by Betting Type')
    ax1.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Break Even')
    ax1.legend()
    ax1.tick_params(axis='x', rotation=45)
    
    # 的中回数
    ax2 = axes[0, 1]
    hit_counts = [best_results[bt]['hit_count'] for bt in bet_types]
    bars2 = ax2.bar(bet_types, hit_counts, alpha=0.7, color='orange')
    ax2.set_ylabel('Hit Count')
    ax2.set_title('Best Hit Count by Betting Type')
    ax2.tick_params(axis='x', rotation=45)
    
    # 評価値
    ax3 = axes[1, 0]
    eval_scores = [best_results[bt]['evaluation_score'] for bt in bet_types]
    bars3 = ax3.bar(bet_types, eval_scores, alpha=0.7, color='green')
    ax3.set_ylabel('Evaluation Score')
    ax3.set_title('Best Evaluation Score by Betting Type')
    ax3.tick_params(axis='x', rotation=45)
    
    # 利益
    ax4 = axes[1, 1]
    profits = [best_results[bt]['total_payout'] - best_results[bt]['total_bet'] for bt in bet_types]
    colors = ['red' if p < 0 else 'blue' for p in profits]
    bars4 = ax4.bar(bet_types, profits, alpha=0.7, color=colors)
    ax4.set_ylabel('Profit (円)')
    ax4.set_title('Best Profit by Betting Type')
    ax4.axhline(y=0, color='black', linestyle='-', alpha=0.7)
    ax4.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # 2. 結果サマリーテーブル
    summary_data = []
    for bet_type in bet_types:
        result = best_results[bet_type]
        profit = result['total_payout'] - result['total_bet']
        summary_data.append({
            'Betting Type': bet_type,
            'Total Bet': f"{result['total_bet']:,}円",
            'Total Payout': f"{result['total_payout']:,}円",
            'Profit': f"{profit:,}円",
            'Recovery Rate': f"{result['recovery_rate']:.3f}",
            'Hit Count': result['hit_count'],
            'Evaluation Score': f"{result['evaluation_score']:.6f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    print("\n=== 全馬券種最適化結果サマリー ===")
    print(summary_df.to_string(index=False))
    
    return summary_df


In [ ]:
# 実行例
print("全馬券種購入シミュレーションを開始...")
print(f"利用可能CPU数: {mp.cpu_count()}")

# 全馬券種のグリッドサーチを実行
best_thresholds_all, best_results_all, all_results_all = grid_search_optimized_betting_types(
    df, odds, haraimodoshi, n_workers=16
)

# 結果表示とプロット
summary_df = plot_all_betting_types_results(best_results_all, all_results_all)

# 各馬券種の最適閾値も表示
print("\n=== 各馬券種の最適閾値 ===")
for bet_type, thresholds in best_thresholds_all.items():
    print(f"\n{bet_type}:")
    for key, value in thresholds.items():
        print(f"  {key}: {value}")

In [ ]:
def analyze_and_plot_all_betting_types_results(all_results_dict):
    """
    全馬券種の結果をグループ化して分析・プロット
    
    Parameters:
    -----------
    all_results_dict : dict
        各馬券種のDataFrameを含む辞書 {'tansho': DataFrame, 'fukusho': DataFrame, ...}
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    
    # 各馬券種で利益と的中率を計算
    processed_results = {}
    
    for bet_type, results_df in all_results_dict.items():
        if len(results_df) == 0:
            continue
            
        # 利益を計算
        results_df['profit'] = results_df['total_payout'] - results_df['total_bet']
        
        # 的中率を計算
        results_df['hit_rate'] = np.where(results_df['race_count'] > 0, 
                                         results_df['hit_count'] / results_df['race_count'], 0)
        
        # グループ化のためのキーを作成
        results_df['group_key'] = (
            results_df['score_0'].astype(str) + '_' +
            results_df['score_1'].astype(str) + '_' +
            results_df['score_2'].astype(str) + '_' +
            results_df['score_ranking_0'].astype(str) + '_' +
            results_df['score_ranking_1'].astype(str) + '_' +
            results_df['score_ranking_2'].astype(str)
        )
        
        processed_results[bet_type] = results_df
    
    # 1. 馬券種別最適パフォーマンス比較
    fig1, axes1 = plt.subplots(2, 2, figsize=(16, 12))
    fig1.suptitle('Best Performance Comparison by Betting Type', fontsize=16, y=0.98)
    
    bet_types = list(processed_results.keys())
    colors = plt.cm.Set3(np.linspace(0, 1, len(bet_types)))
    
    # 最高回収率
    ax1 = axes1[0, 0]
    max_recovery_rates = [processed_results[bt]['recovery_rate'].max() for bt in bet_types]
    bars1 = ax1.bar(bet_types, max_recovery_rates, color=colors, alpha=0.7)
    ax1.set_ylabel('Max Recovery Rate')
    ax1.set_title('Maximum Recovery Rate by Betting Type')
    ax1.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Break Even')
    ax1.legend()
    ax1.tick_params(axis='x', rotation=45)
    
    # 値をバーの上に表示
    for i, v in enumerate(max_recovery_rates):
        ax1.text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom')
    
    # 最高的中回数
    ax2 = axes1[0, 1]
    max_hit_counts = [processed_results[bt]['hit_count'].max() for bt in bet_types]
    bars2 = ax2.bar(bet_types, max_hit_counts, color=colors, alpha=0.7)
    ax2.set_ylabel('Max Hit Count')
    ax2.set_title('Maximum Hit Count by Betting Type')
    ax2.tick_params(axis='x', rotation=45)
    
    # 値をバーの上に表示
    for i, v in enumerate(max_hit_counts):
        ax2.text(i, v + 0.1, f'{int(v)}', ha='center', va='bottom')
    
    # 最高利益
    ax3 = axes1[1, 0]
    max_profits = [processed_results[bt]['profit'].max() for bt in bet_types]
    profit_colors = ['red' if p < 0 else 'blue' for p in max_profits]
    bars3 = ax3.bar(bet_types, max_profits, color=profit_colors, alpha=0.7)
    ax3.set_ylabel('Max Profit (円)')
    ax3.set_title('Maximum Profit by Betting Type')
    ax3.axhline(y=0, color='black', linestyle='-', alpha=0.7)
    ax3.tick_params(axis='x', rotation=45)
    
    # 値をバーの上に表示
    for i, v in enumerate(max_profits):
        ax3.text(i, v + max(max_profits) * 0.01, f'{int(v):,}', ha='center', va='bottom')
    
    # 最高評価値
    ax4 = axes1[1, 1]
    max_eval_scores = [processed_results[bt]['evaluation_score'].max() for bt in bet_types]
    bars4 = ax4.bar(bet_types, max_eval_scores, color=colors, alpha=0.7)
    ax4.set_ylabel('Max Evaluation Score')
    ax4.set_title('Maximum Evaluation Score by Betting Type')
    ax4.tick_params(axis='x', rotation=45)
    
    # 値をバーの上に表示
    for i, v in enumerate(max_eval_scores):
        ax4.text(i, v + max(max_eval_scores) * 0.01, f'{v:.4f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # 2. 各馬券種のオッズ閾値別パフォーマンス
    fig2, axes2 = plt.subplots(2, 2, figsize=(18, 12))
    fig2.suptitle('Performance vs Min Odds Threshold by Betting Type', fontsize=16, y=0.98)
    
    # 回収率 vs オッズ閾値
    ax1 = axes2[0, 0]
    for i, (bet_type, results_df) in enumerate(processed_results.items()):
        recovery_by_odds = results_df.groupby('min_odds')['recovery_rate'].mean().reset_index()
        ax1.plot(recovery_by_odds['min_odds'], recovery_by_odds['recovery_rate'], 
                marker='o', linewidth=2, markersize=4, 
                color=colors[i], label=bet_type, alpha=0.8)
    
    ax1.set_xlabel('Min Odds Threshold', fontsize=11)
    ax1.set_ylabel('Average Recovery Rate', fontsize=11)
    ax1.set_title('Recovery Rate vs Min Odds Threshold', fontsize=12)
    ax1.grid(True, alpha=0.3)
    ax1.axhline(y=1.0, color='red', linestyle='--', alpha=0.7)
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # 的中率 vs オッズ閾値
    ax2 = axes2[0, 1]
    for i, (bet_type, results_df) in enumerate(processed_results.items()):
        hit_rate_by_odds = results_df.groupby('min_odds')['hit_rate'].mean().reset_index()
        ax2.plot(hit_rate_by_odds['min_odds'], hit_rate_by_odds['hit_rate'] * 100, 
                marker='s', linewidth=2, markersize=4, 
                color=colors[i], label=bet_type, alpha=0.8)
    
    ax2.set_xlabel('Min Odds Threshold', fontsize=11)
    ax2.set_ylabel('Average Hit Rate (%)', fontsize=11)
    ax2.set_title('Hit Rate vs Min Odds Threshold', fontsize=12)
    ax2.grid(True, alpha=0.3)
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # 平均利益 vs オッズ閾値
    ax3 = axes2[1, 0]
    for i, (bet_type, results_df) in enumerate(processed_results.items()):
        profit_by_odds = results_df.groupby('min_odds')['profit'].mean().reset_index()
        ax3.plot(profit_by_odds['min_odds'], profit_by_odds['profit'], 
                marker='^', linewidth=2, markersize=4, 
                color=colors[i], label=bet_type, alpha=0.8)
    
    ax3.set_xlabel('Min Odds Threshold', fontsize=11)
    ax3.set_ylabel('Average Profit (円)', fontsize=11)
    ax3.set_title('Profit vs Min Odds Threshold', fontsize=12)
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # レース数 vs オッズ閾値
    ax4 = axes2[1, 1]
    for i, (bet_type, results_df) in enumerate(processed_results.items()):
        race_count_by_odds = results_df.groupby('min_odds')['race_count'].mean().reset_index()
        ax4.plot(race_count_by_odds['min_odds'], race_count_by_odds['race_count'], 
                marker='D', linewidth=2, markersize=4, 
                color=colors[i], label=bet_type, alpha=0.8)
    
    ax4.set_xlabel('Min Odds Threshold', fontsize=11)
    ax4.set_ylabel('Average Race Count', fontsize=11)
    ax4.set_title('Race Count vs Min Odds Threshold', fontsize=12)
    ax4.grid(True, alpha=0.3)
    ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()
    
    # 3. 馬券種別詳細統計テーブル
    summary_stats = []
    
    for bet_type, results_df in processed_results.items():
        stats = {
            'Betting Type': bet_type,
            'Max Recovery Rate': f"{results_df['recovery_rate'].max():.3f}",
            'Avg Recovery Rate': f"{results_df['recovery_rate'].mean():.3f}",
            'Max Hit Count': int(results_df['hit_count'].max()),
            'Avg Hit Count': f"{results_df['hit_count'].mean():.1f}",
            'Max Profit': f"{results_df['profit'].max():,.0f}円",
            'Avg Profit': f"{results_df['profit'].mean():,.0f}円",
            'Max Evaluation Score': f"{results_df['evaluation_score'].max():.6f}",
            'Total Combinations': len(results_df)
        }
        summary_stats.append(stats)
    
    summary_df = pd.DataFrame(summary_stats)
    
    print("\n=== 全馬券種統計サマリー ===")
    print(summary_df.to_string(index=False))
    
    # 4. 各馬券種の上位パラメータ組み合わせ
    print(f"\n=== 各馬券種の上位3パラメータ組み合わせ ===")
    
    top_params_all = {}
    
    for bet_type, results_df in processed_results.items():
        print(f"\n【{bet_type}】")
        top_3 = results_df.nlargest(3, 'evaluation_score')
        
        for idx, (_, row) in enumerate(top_3.iterrows(), 1):
            print(f"  {idx}位: 評価値={row['evaluation_score']:.6f}")
            print(f"    スコア閾値: ({row['score_0']:.2f}, {row['score_1']:.2f}, {row['score_2']:.2f})")
            print(f"    ランク閾値: ({int(row['score_ranking_0'])}, {int(row['score_ranking_1'])}, {int(row['score_ranking_2'])})")
            print(f"    最小オッズ: {row['min_odds']:.1f}")
            print(f"    回収率: {row['recovery_rate']:.3f}, 的中: {int(row['hit_count'])}回, 利益: {int(row['profit']):,}円")
        
        top_params_all[bet_type] = top_3
    
    # 5. ヒートマップ（馬券種別 × 評価指標）
    fig3, ax = plt.subplots(figsize=(12, 8))
    
    heatmap_data = []
    metrics = ['recovery_rate', 'hit_count', 'profit', 'evaluation_score']
    metric_names = ['Recovery Rate', 'Hit Count', 'Profit', 'Evaluation Score']
    
    for bet_type in bet_types:
        row = []
        results_df = processed_results[bet_type]
        row.append(results_df['recovery_rate'].max())
        row.append(results_df['hit_count'].max())
        row.append(results_df['profit'].max())
        row.append(results_df['evaluation_score'].max())
        heatmap_data.append(row)
    
    heatmap_df = pd.DataFrame(heatmap_data, index=bet_types, columns=metric_names)
    
    # 各列を正規化（0-1スケール）
    heatmap_normalized = heatmap_df.div(heatmap_df.max())
    
    sns.heatmap(heatmap_normalized, annot=True, fmt='.3f', cmap='YlOrRd', 
                cbar_kws={'label': 'Normalized Score'}, ax=ax)
    ax.set_title('Performance Heatmap (Normalized Max Values)', fontsize=14)
    ax.set_xlabel('Performance Metrics', fontsize=12)
    ax.set_ylabel('Betting Types', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    return processed_results, summary_df, top_params_all

def create_betting_type_comparison_chart(best_results_all):
    """
    馬券種別の最適結果比較チャート
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Optimal Results Comparison by Betting Type', fontsize=16, y=0.98)
    
    bet_types = list(best_results_all.keys())
    colors = plt.cm.tab10(np.linspace(0, 1, len(bet_types)))
    
    # 1. 回収率
    ax1 = axes[0, 0]
    recovery_rates = [best_results_all[bt]['recovery_rate'] for bt in bet_types]
    bars1 = ax1.bar(bet_types, recovery_rates, color=colors, alpha=0.7)
    ax1.set_ylabel('Recovery Rate')
    ax1.set_title('Recovery Rate')
    ax1.axhline(y=1.0, color='red', linestyle='--', alpha=0.7)
    ax1.tick_params(axis='x', rotation=45)
    
    # 2. 的中回数
    ax2 = axes[0, 1]
    hit_counts = [best_results_all[bt]['hit_count'] for bt in bet_types]
    bars2 = ax2.bar(bet_types, hit_counts, color=colors, alpha=0.7)
    ax2.set_ylabel('Hit Count')
    ax2.set_title('Hit Count')
    ax2.tick_params(axis='x', rotation=45)
    
    # 3. 利益
    ax3 = axes[0, 2]
    profits = [best_results_all[bt]['total_payout'] - best_results_all[bt]['total_bet'] for bt in bet_types]
    profit_colors = ['red' if p < 0 else 'blue' for p in profits]
    bars3 = ax3.bar(bet_types, profits, color=profit_colors, alpha=0.7)
    ax3.set_ylabel('Profit (円)')
    ax3.set_title('Profit')
    ax3.axhline(y=0, color='black', linestyle='-', alpha=0.7)
    ax3.tick_params(axis='x', rotation=45)
    
    # 4. 購入総額
    ax4 = axes[1, 0]
    total_bets = [best_results_all[bt]['total_bet'] for bt in bet_types]
    bars4 = ax4.bar(bet_types, total_bets, color=colors, alpha=0.7)
    ax4.set_ylabel('Total Bet (円)')
    ax4.set_title('Total Bet Amount')
    ax4.tick_params(axis='x', rotation=45)
    
    # 5. 払戻総額
    ax5 = axes[1, 1]
    total_payouts = [best_results_all[bt]['total_payout'] for bt in bet_types]
    bars5 = ax5.bar(bet_types, total_payouts, color=colors, alpha=0.7)
    ax5.set_ylabel('Total Payout (円)')
    ax5.set_title('Total Payout Amount')
    ax5.tick_params(axis='x', rotation=45)
    
    # 6. 評価値
    ax6 = axes[1, 2]
    eval_scores = [best_results_all[bt]['evaluation_score'] for bt in bet_types]
    bars6 = ax6.bar(bet_types, eval_scores, color=colors, alpha=0.7)
    ax6.set_ylabel('Evaluation Score')
    ax6.set_title('Evaluation Score')
    ax6.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return fig

In [ ]:
# 全馬券種のシミュレーション結果を分析
processed_results, summary_df, top_params_all = analyze_and_plot_all_betting_types_results(all_results_all)

# 最適結果の比較チャートも作成
comparison_chart = create_betting_type_comparison_chart(best_results_all)

# CSVファイルに結果を保存
summary_df.to_csv('betting_types_summary.csv', index=False, encoding='utf-8-sig')

# 各馬券種の詳細結果もCSVで保存
for bet_type, results_df in processed_results.items():
    results_df.to_csv(f'{bet_type}_detailed_results.csv', index=False, encoding='utf-8-sig')

print("分析完了！結果ファイルを保存しました。")